In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/ImageSearchDemo'
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ImageSearchDemo
display_results_colab.py  Mobile/          saved_model     train/
expected_images_data/     npy_files/       search.py       train.ipynb
faiss_data/               __pycache__/     server/         variables/
image_utils.py            resnet50.tflite  test.ipynb      wikiart_search_data/
input_images/             results/         test_search.py


In [ ]:
import tensorflow as tf 
import numpy as np

In [ ]:
base_model = tf.keras.applications.ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpsds6wg1o/assets


INFO:tensorflow:Assets written to: /tmp/tmpsds6wg1o/assets


In [ ]:
with open("resnet50_float16.tflite", 'wb') as f:
  f.write(tflite_model)

In [ ]:
TFLITE_MODEL = "resnet50.tflite"

In [ ]:
# Load TFLite model and see some details about input/output
tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
name: serving_default_input_2:0
shape: [  1 224 224   3]
type: <class 'numpy.float32'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [   1    7    7 2048]
type: <class 'numpy.float32'>
== Input details ==
name: serving_default_input_2:0
shape: [  1 224 224   3]
type: <class 'numpy.float32'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [   1    7    7 2048]
type: <class 'numpy.float32'>


In [ ]:
tflite_interpreter.allocate_tensors()

In [ ]:
tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

input =  np.random.uniform(low=0, high=1, size=(1, 224, 224, 3))
input = input.astype(np.float32)
print(input[0, 0:2])
tflite_interpreter.set_tensor(input_details[0]['index'], input)

tflite_interpreter.invoke()

tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
print("mean", np.mean(tflite_model_predictions))
print("std", np.std(tflite_model_predictions))
print("Predictions", tflite_model_predictions[0, 0:2])
tflite_interpreter.reset_all_variables()

[[[0.03259031 0.28605646 0.34830928]
  [0.05155633 0.11687771 0.9969602 ]
  [0.8743616  0.03903335 0.20985809]
  ...
  [0.84562165 0.69598347 0.81291544]
  [0.25915253 0.297984   0.554626  ]
  [0.79028904 0.59814817 0.19536051]]

 [[0.9677556  0.37637478 0.05428945]
  [0.54587346 0.8406689  0.08144191]
  [0.01415316 0.24027169 0.6700371 ]
  ...
  [0.20200542 0.8394572  0.8618907 ]
  [0.877701   0.48458943 0.71056724]
  [0.6269466  0.76316226 0.9320616 ]]]
mean 0.102766074
std 0.7924372
Predictions [[[0.         0.         0.         ... 2.335537   0.         0.        ]
  [0.         0.         0.         ... 1.6103444  0.         0.        ]
  [0.         0.         0.         ... 1.0865581  0.         0.        ]
  ...
  [0.         0.         0.         ... 1.8970535  0.         0.        ]
  [0.         0.         0.         ... 1.62458    0.         0.        ]
  [0.         0.         0.         ... 2.9182153  0.         0.        ]]

 [[0.         0.         0.         ... 0.   